### *Widget - SIR/SIS/etc..*

In [7]:
import matplotlib.pyplot as plt
import numpy as np
import bqplot
from ipywidgets import widgets, Layout
from IPython.display import display, clear_output

In [2]:
class SIR:
    """
    SIR and SIRV model. SIR if mu=0 and SIRV if mu>0
    """
    
    def __init__(self, s=0.9, i=0.1, r=0.0, lamda=1, gamma=0.5, mu=0):
        # Anfangswerte
        self.S_init = s
        self.I_init = i
        self.R_init = r
        # Parameter
        self.lamda = lamda
        self.gamma = gamma
        self.mu = mu
    
    # *****Differentialgleichungen*****
    def delta_S(self, S, I):
        return -self.lamda*S*I + self.mu - self.mu*S

    def delta_I(self, S, I):
        return self.lamda*S*I - self.gamma*I-self.mu*I

    def delta_R(self, I, R):
        return self.gamma*I - self.mu*R

    def delta_SIR(self, t, SIR):
        return np.array([self.delta_S(SIR[0],SIR[1]), self.delta_I(SIR[0],SIR[1]),self.delta_R(SIR[1], SIR[2])])
    # **********************************
    
    #* **********Loesen des AWP**********
    def compute(self, t):
        """
        Berechnet die Loesung des Anfangswertproblems bis zu einem gegebenen Zeitpunkt t.
        Dazu nutzt die Funktion das Runge Kutte Loesungsverfahren.
        """
        self.stepsize = 0.5
        time = np.arange(0,t,self.stepsize)
        initial_values = np.array([self.S_init, self.I_init, self.R_init])
        solution = np.zeros((len(time), len(initial_values)))
        solution[0] = initial_values
        # Die Berechnung erfolgt entsprechend des Runge Kutta Schemas vierter Ordnung
        for i in range(1, len(time)):
            t_last = time[i-1]
            t = time[i]
            tau = t-t_last
            # Der letzte berechnete Wert von u
            solution_last = solution[i-1]
            # Berechnung von k1
            k1 = self.delta_SIR(t_last, solution_last)
            # Berechnung von k2
            k2 = self.delta_SIR(t_last + tau/2, solution_last+tau/2*k1)
            # Berechnung von k3
            k3 = self.delta_SIR(t_last + tau/2, solution_last+tau/2*k2)
            # Berechnung von k4
            k4 = self.delta_SIR(t, solution_last+tau*k3)
            # Berechnung des neuen Gesamtwertes
            solution_new = solution_last + tau * (1/6) * (k1+2*k2+2*k3+k4)
            # Speichern des neuen Werts
            solution[i] = solution_new
        return tuple(solution.T)#(solution[:,0], solution[:,1], solution[:,2])
    # **********************************

In [3]:
class SIS:
    """
    SIS Model
    """
    
    def __init__(self, s=0.9, i=0.1, lamda=1, gamma=0.5, mu=1):
        # Anfangswerte
        self.S_init = s
        self.I_init = i
        # Parameter
        self.lamda = lamda
        self.gamma = gamma
        self.mu = mu
        
    # *****Differentialgleichungen*****
    def delta_S(self, S, I):
        return -self.lamda*I*S + self.gamma*I + self.mu - self.mu*S
    
    def delta_I(self, S, I):
        return self.lamda*I*S - self.gamma*I - self.mu*I
        
    def delta_SIS(self, t, SIS):
        return np.array([self.delta_S(SIS[0],SIS[1]), self.delta_I(SIS[0],SIS[1])])
    # **********************************
    
    
    #* **********Loesen des AWP**********
    def compute(self, t):
        """
        Berechnet die Loesung des Anfangswertproblems bis zu einem gegebenen Zeitpunkt t.
        Dazu nutzt die Funktion das Runge Kutte Loesungsverfahren.
        """
        self.stepsize = 0.5
        time = np.arange(0,t,self.stepsize)
        initial_values = np.array([self.S_init, self.I_init])
        solution = np.zeros((len(time), len(initial_values)))
        solution[0] = initial_values
        # Die Berechnung erfolgt entsprechend des Runge Kutta Schemas vierter Ordnung
        for i in range(1, len(time)):
            t_last = time[i-1]
            t = time[i]
            tau = t-t_last
            # Der letzte berechnete Wert von u
            solution_last = solution[i-1]
            # Berechnung von k1
            k1 = self.delta_SIS(t_last, solution_last)
            # Berechnung von k2
            k2 = self.delta_SIS(t_last + tau/2, solution_last+tau/2*k1)
            # Berechnung von k3
            k3 = self.delta_SIS(t_last + tau/2, solution_last+tau/2*k2)
            # Berechnung von k4
            k4 = self.delta_SIS(t, solution_last+tau*k3)
            # Berechnung des neuen Gesamtwertes
            solution_new = solution_last + tau * (1/6) * (k1+2*k2+2*k3+k4)
            # Speichern des neuen Werts
            solution[i] = solution_new
        return tuple(solution.T)
    # **********************************

In [111]:
class EpiModWidget:
    """
    Interaktives Widget zur Demonstration unterschiedlicher infektionsepidemiologischer Modelle.
    """
    
    def __init__(self, model='SIR'):
        """
        Initialisierung des Widgets.
        """
        # Subwidgets
        self.lamda = widgets.FloatSlider(1, min=0.01, max=10, description='\N{greek small letter lamda}')
        # Daily recovery removal rate
        self.gamma = widgets.FloatSlider(0.5, min=0.01, max=10, description='\N{greek small letter gamma}')
        # Daily death removal rate
        self.mu = widgets.FloatSlider(0.05, min=0.01, max=1, description='\N{greek small letter mu}')
        # Daily reproduction rate
        sigma = str(round(self.lamda.value/(self.gamma.value+self.mu.value),2))
        self.sigma = widgets.Label(sigma)
        # Sum of initial values
        self.sum = widgets.Label(str(0))

        
        self.s_init = widgets.FloatSlider(0.9, min=0, max=1, description="S(0)")
        self.i_init = widgets.FloatSlider(0.1, min=0, max=1, description="I(0)")
        self.r_init = widgets.FloatSlider(0, min=0, max=1, description="R(0)")
        
        self.htitle = "Epidemiologische Modellierung"
        
        self.button_redraw = widgets.Button(description='Redraw')
        
        self.label = widgets.HTML(value='')
        
        self.tab = widgets.Tab(children=[widgets.VBox([widgets.Label("Parameter:"),self.lamda, self.gamma, self.mu, 
                                                       widgets.VBox([self.sigma],layout=Layout(width='100%', display='flex',
                                                                                              align_items='center')),
                                                       widgets.Label("Anfangswerte: "), self.s_init, self.i_init, self.r_init,
                                                       widgets.VBox([self.sum],layout=Layout(width='100%', display='flex',
                                                                                              align_items='center')),]),
                                         widgets.VBox([widgets.Label("Parameter:"), self.lamda, self.gamma,
                                                       widgets.VBox([self.sigma],layout=Layout(width='100%', display='flex',
                                                                                              align_items='center')),
                                                       widgets.Label("Anfangswerte:"), 
                                                       self.s_init, self.i_init, self.r_init, 
                                                      widgets.VBox([self.sum],layout=Layout(width='100%', display='flex',
                                                                                              align_items='center')),]),
                                         widgets.VBox([widgets.Label("Parameter:"), self.lamda, self.gamma, self.mu, 
                                                       widgets.VBox([self.sigma],layout=Layout(width='100%', display='flex',
                                                                                              align_items='center')),
                                                                     widgets.Label("Anfangswerte:"),
                                                                     self.s_init, self.i_init,
                                                      widgets.VBox([self.sum],layout=Layout(width='100%', display='flex',
                                                                                              align_items='center')),])],
                               _titles = {'0': 'SIRV', '1': 'SIR', '2': 'SIS'})
        
        # Choose the model
        if model=='SIRV':
            self.tab.selected_index='0'
        elif model=='SIR':
            self.tab.selected_index='1'
        elif model=='SIS':
            self.tab.selected_index='2'
        else:
            raise ValueError('Unbekanntes Modell. Zur Verfuegung stehen SIRV/SIR/SIS')
        
        # Create canvas elements
        self.canvas_layout = widgets.Layout(width='640px', height='640')
        self.sc_x = bqplot.LinearScale()
        self.sc_y = bqplot.LinearScale(min=0, max=1.2)
        self.S_line = bqplot.Lines(x=[], y=[], scales = {'x': self.sc_x, 'y':self.sc_y}, display_legend=True,
                                   labels=['Susceptible'], colors=['green'])
        self.I_line = bqplot.Lines(x=[], y=[], scales = {'x': self.sc_x, 'y':self.sc_y}, display_legend=True,
                                   labels=['Infected'], colors=['blue'])
        self.R_line = bqplot.Lines(x=[], y=[], scales = {'x': self.sc_x, 'y':self.sc_y}, display_legend=True,
                                   labels=['Removed'], colors=['red'])
        self.ax_x = bqplot.Axis(scale=self.sc_x, label='t')
        self.ax_y = bqplot.Axis(scale=self.sc_y,orientation='vertical', label='')
        self.new_canvas()
        
        # Set callback functions
        self.button_redraw.on_click(self.redraw_function)
        self.lamda.observe(self.update_lines, names='value')
        self.gamma.observe(self.update_lines, names='value')
        self.mu.observe(self.update_lines, names='value')
        self.s_init.observe(self.update_lines, names='value')
        self.i_init.observe(self.update_lines, names='value')
        self.r_init.observe(self.update_lines, names='value')
        self.tab.observe(self.update_all, names='selected_index')
        
    def show(self):
        """Anzeigen des Widgets."""
        display(self.ui, display_id='ui')
        

    def get_model(self):
        if self.tab.selected_index == 0:
            pass
        elif self.tab.selected_index == 1:
            model = SIR(s=self.s_init.value, i=self.i_init.value, r=self.r_init.value,
                        lamda=self.lamda.value, gamma=self.gamma.value)
            
        else:
            pass
        return model
            
    def new_canvas(self):
        """Erstellen einer neuen Leinwand."""
        self.update_lines()
        if self.tab.selected_index == 0:
            # SIRV
            self.canvas = bqplot.Figure(marks=[self.S_line, self.I_line, self.R_line],
                                        axes=[self.ax_x, self.ax_y],
                                        title="SIRV", 
                                        layout=self.canvas_layout,
                                       legend_location='top-right')
        elif self.tab.selected_index == 1:
            # SIR
            self.canvas = bqplot.Figure(marks=[self.S_line, self.I_line, self.R_line],
                                        axes=[self.ax_x, self.ax_y],
                                        title="SIR", 
                                        layout=self.canvas_layout,
                                       legend_location='top-right')
        else:
            # SIS
            self.canvas = bqplot.Figure(marks=[self.S_line, self.I_line],
                                        axes=[self.ax_x, self.ax_y],
                                        title="SIS", 
                                        layout=self.canvas_layout,
                                        legend_location='top-right')
        
        self.ui = widgets.VBox([widgets.HBox([self.canvas, widgets.VBox([self.tab, self.button_redraw])]), 
                               self.label])

    def update_all(self, change=None):
        clear_output(wait=True)
        self.sigma.value = ('\N{greek small letter mu} = ' + 
                            str(round(self.lamda.value/(self.mu.value+self.gamma.value),2)))
        self.new_canvas()
        self.show()

    def update_lines(self, change=None):
        time = np.linspace(0,100,100)
        if self.tab.selected_index == 0:
            # SIRV
            self.sigma.value = ('\N{greek small letter sigma} = ' + 
                            str(round(self.lamda.value/(self.mu.value+self.gamma.value),2)))
            self.sum.value = ("S(0)+I(0)+R(0) = " + str(self.s_init.value+self.i_init.value+self.r_init.value))
            model = SIR(self.s_init.value, self.i_init.value, self.r_init.value,
                         self.lamda.value, self.gamma.value, self.mu.value)
            self.S_line.x, self.I_line.x, self.R_line.x = tuple([time]*3)
            solution = model.compute(100)
            self.S_line.y, self.I_line.y, self.R_line.y = solution
        elif self.tab.selected_index == 1:
            # SIR
            self.sigma.value = ('\N{greek small letter sigma} = ' + 
                            str(round(self.lamda.value/self.gamma.value,2)))
            self.sum.value = ("S(0)+I(0)+R(0) = " + str(self.s_init.value+self.i_init.value+self.r_init.value))
            model = SIR(self.s_init.value, self.i_init.value, self.r_init.value,
                         self.lamda.value, self.gamma.value)
            self.S_line.x, self.I_line.x, self.R_line.x = tuple([time]*3)
            solution = model.compute(100)
            self.S_line.y, self.I_line.y, self.R_line.y = solution 
        else:
            #SIS
            self.sigma.value = ('\N{greek small letter sigma} = ' + 
                            str(round(self.lamda.value/(self.mu.value+self.gamma.value),2)))
            self.sum.value = ("S(0)+I(0) = " + str(self.s_init.value+self.i_init.value))
            model = SIS(self.s_init.value, self.i_init.value, self.lamda.value, self.gamma.value,
                       self.mu.value)
            self.S_line.x, self.I_line.x = tuple([time]*2)
            solution = model.compute(100)
            self.S_line.y, self.I_line.y = solution

    def redraw_function(self, button):
        self.update_all()
            

In [112]:
em = EpiModWidget()
em.show()